## 4.7 前向传播、反向传播和计算图
我们已经学习了如何用小批量随机梯度下降训练模型。然而当实现该算法时，我们只考虑了通过前向传播(forward propagation)所涉及的计算。在计算梯度时，我们只调用了深度学习提供的反向传播函数，而不知其所以然。

梯度的自动计算(自动微分)大大简化了深度学习算法的实现。在自动微分之前，即使是对复杂模型的微小调整也需要手工重新计算复杂的导数，学术论文也不得不分配大量页面来推导更新规则。本节中，我们将通过一些基本的数学和计算图，深入探讨反向传播的细节。首先，我们将重点放在带权重衰减($L_2$正则化)的单隐藏层多层感知机上。


### 4.7.1 前向传播
前向传播(forward propagation或forward pass)指的是：按顺序(从输入层到输出层)计算和存储神经网络中每层的结果

我们将一步步研究单隐藏层神经网络的机制，我们假设输入样本是$x \in \mathbb R^d$，并且我们的隐藏层不包括偏置项。这里的中间变量是：
$$
z = W^{(1)}x
$$
其中$W^{(1)} \in \mathbb R^{h \times d}$是隐藏层的权重参数。将中间变量$z \in \mathbb R^h$通过激活函数$\phi$后，我们得到长度为$h$的隐藏激活向量：
$$
h = \phi (z)
$$
隐藏变量$h$也是一个中间变量。假设输出层的参数只有权重$W^{(2)} \in \mathbb R^{q \times h}$，我们可以得到输出层变量，它是一个长度为$q$的向量：
$$
o = W^{(2)} h
$$

假设损失函数为$l$，样本标签$y$，我们可以计算单个数据样本的损失项，
$$
L = l(o, y)
$$
根据$L_2$正则化的定义，给定超参数$\lambda$，正则化项为
$$
s = \frac{\lambda}{2} (\mid \mid W^{(1)}\mid \mid ^2 _F + \mid \mid W^{(2)} \mid \mid ^2 _F)
$$

其中矩阵的Frobenius范数是将矩阵展平为向量后应用的$L_2$范数。最后，模型在给定模型数据样本上的正则化损失为：
$$
J = L + s
$$
在下面的讨论中，我们将$J$称为目标函数(objective function)。

### 4.7.2 前向传播计算图
绘制计算图有助于我们可视化计算中操作符和变量的依赖关系。下图是与上述简单网络相对应的计算图，其中正方形表示变量，圆圈表示操作符。左下角表示输入，右下角表示输出。注意显示数据流的箭头方向主要是向右和向上。

<div align=center>
<img src='../../pics/4_7_1.jpeg' width='50%'>
</div>

### 4.7.3 反向传播
反向传播(backward propagation或backpropagattin)指的是计算神经网络参数梯度的方法。该方法根据微积分中的链式法则，按相反的顺序从输出层到输入层遍历网络。该算法存储了计算某些参数梯度时所需的任何中间变量(偏导数)。假设我们有函数$Y=f(X)$和$Z=g(Y)$，其中输入和输出$X, Y, Z$是任意形状的张量。利用链式法则，我们可以计算$Z$关于$X$的导数：
$$
\frac{\partial Z}{\partial X} = prod(\frac{\partial Z}{\partial Y}, \frac {\partial Y}{\partial X})
$$

在这里，我们使用prod运算符在执行必要的操作(如换位和交换输入位置)后将其参数相乘。对于向量，它只是矩阵-矩阵乘法。对于高维张量，我们使用适当的对应项。运算符prod指代了所有的这些符号。

回想一下，在计算图 `图4.7.1`中的单隐藏层简单网络的参数是$W^{(1)}$和$W^{(2)}$。反向传播的目的是计算梯度$\partial J/\partial W^{(1)}$和$\partial J/\partial W^{(2)}$。为此，我们应用链式法则，依次计算每个中间变量和参数的梯度。计算的顺序与前向传播中执行的顺序相反，因为我们需要从计算图的结果开始，并朝着参数的方向努力。第一步是计算目标函数$J = L + s$相对于损失项$L$和正则项$s$的梯度。

$$
\frac{\partial J}{\partial L} = 1 and \frac{\partial J}{\partial s} = 1
$$

接下来，我们根据链式法则计算目标函数关于输出层变量$o$的梯度：
$$
\frac{\partial J}{\partial o} = prod(\frac{\partial J}{\partial L}, \frac{\partial L}{\partial o}) = \frac{\partial L}{\partial o} \in \mathbb R^q
$$

我们再计算正则化项相对于两个参数的梯度：
$$
\frac{\partial s}{\partial W^{(1)}} = \lambda W^{(1)} and \frac{\partial s}{\partial W^{(2)}} = \lambda W^{(2)}
$$

我们可以计算最接近输出层的模型参数的梯度$\partial J/ \partial W^{(2)} \in \mathbb R^{q \times h}$。使用链式法则得出：
$$
\frac{\partial J}{\partial W^{(2)}} = prod(\frac{\partial J}{\partial o}, \frac{\partial o}{\partial W^{(2)}}) + prod(\frac{\partial J}{\partial s}, \frac{\partial s}{\partial W^{(2)}}) = \frac{\partial J}{\partial o} h^T + \lambda W^{(2)}
$$

为了获得关于$W^{(1)}$的梯度，我们需要继续沿着输出层到隐藏层反向传播。关于隐藏层输出的梯度$\partial J/\partial h \in \mathbb R^h$由下式给出：
$$
\frac{\partial J}{\partial h} = prod(\frac{\partial J}{\partial o}, \frac{\partial o}{\partial h}) = {W^{(2)}}^T \frac{\partial J}{\partial o}
$$

由于激活函数$\phi$是按元素计算的，计算中间变量$z$的梯度$\partial J / \partial z \in \mathbb R^h$需要使用按元素乘法运算符，我们用$\odot$表示：
$$
\frac {\partial J}{\partial z} = prod (\frac{\partial J}{\partial h}, \frac{\partial h}{\partial z})=\frac{\partial J}{\partial h} \odot \phi'(z)
$$

最后我们可以得到：
$$
\frac{\partial J}{\partial W^{(1)}} = prod(\frac{\partial J}{\partial z}, \frac{\partial z}{\partial W^{(1)}}) + prod(\frac{\partial J}{\partial s}, \frac{\partial s}{\partial W^{(1)}}) = \frac{\partial J}{\partial z} x^T + \lambda W^{(1)}
$$

### 4.7.4 训练神经网络
在训练神经网络时，前向传播和反向传播相互依赖。对于前向传播，我们沿着依赖的方向遍历计算图并计算其路径上的所有变量。然后将这些用于反向传播，其中计算顺序与计算图的相反。

以上述简单网络为例:一方面，在前向传播期间计算正则项取决于模型参数$W^{(1)}$和$W^{(2)}$的当前值。它 们是由优化算法根据最近迭代的反向传播给出的。另一方面，反向传播期间参数的梯度计算，取决于由前向传播给出的隐藏变量$h$的当前值。

因此，在训练神经网络时，在初始化模型参数后，我们交替使用前向传播和反向传播，利用反向传播给出的 梯度来更新模型参数。注意，反向传播重复利用前向传播中存储的中间值，以避免重复计算。带来的影响之 一是我们需要保留中间值，直到反向传播完成。这也是训练比单纯的预测需要更多的内存(显存)的原因之 一。此外，这些中间值的大小与网络层的数量和批量的大小大致成正比。因此，使用更大的批量来训练更深 层次的网络更容易导致内存不足(out of memory)错误。

### 4.7.5 小结
- 前向传播在神经网络定义的计算图中按顺序计算和存储中间变量，它的顺序是从输入层到输出层
- 反向传播按相反的顺序(按输出层到输入层)计算和存储神经网络的中间变量和参数的梯度
- 在训练深度学习模型时，前向传播和反向传播是相互依赖的
- 训练比预测需要更多的内存